In [ ]:
#!/usr/bin/env


"""


make line datacubes and integrated maps
and compare applying continuum selfcal
NOTE: this is intended to be an interactive, iterative process
      so this is more a log that should be run by cutting and
      pasting into casa rather than as an executable script
      search "CHANGEME" for variables to be changed
10/9/15 MCA
"""


#  Setup 

III_1007  M0.0      II


In [ ]:
field = 5                                     # CHANGEME 
fitspw    = '2,3,4,7,8,9'                                   # line-free channels for fitting continuum
linespw   = '0,1,3,5,6,8'                                   # line spectral windows (C18O, 13CO, CN)
robust     = 0.5                                            # CHANGEME
imsize     = [640,640]
cell       = '0.03arcsec'
imagermode = 'csclean'
outframe   = 'lsrk'
veltype    = 'radio'
width      = '1.0km/s'
start      = '-3km/s'
nchan      = 16
xc         = 303                               # CHANGEME
yc         = 300                               # CHANGEME
in_a       = 80
out_a      = 120
aper       = 0.5
boxwidth = 300.
box = rg.box([xc-boxwidth,yc-boxwidth],[xc+boxwidth,yc+boxwidth])


#  Create continuum subtracted line datasets 

In [ ]:
uvcontsub(vis       = 'f'+str(field)+'.vis',                # full vis file for this field
          spw       = linespw,                              # line spw (for cont subtraction)
          fitspw    = fitspw,                               # cont spw
          combine   = 'spw',
          solint    = 'int',
          fitorder  = 1,
          want_cont = False)                                # should not be changed.


#  13CO line 

first try on 13CO line as thats going to be the brightest
os.system('rm -rf f'+str(field)+'_13co32*')


In [ ]:
clean(vis           = 'f'+str(field)+'.vis.contsub',
      imagename     = 'f'+str(field)+'_13co32',
      mode          = 'velocity',
      start         = start,
      width         = width,
      nchan         = nchan,
      outframe      = outframe,
      veltype       = veltype,
      restfreq      = '330.58797GHz',
      niter         = 500,
      threshold     = 0,
      interactive   = False,
      mask          = 'f5_cont_mask.crtf',
      imsize        = imsize,
      cell          = cell,
      weighting     ='briggs',
      robust        = robust,
      imagermode    = imagermode)


used continuum region in automated mode
can't see emission
export cube to fits file    


In [ ]:
fbase = 'f'+str(field)+'_13co32'
os.system('rm -rf '+fbase+'.cube.fits')
exportfits(imagename=fbase+'.image',fitsimage=fbase+'.cube.fits')


use viewer to check channel maps and spectrum
make sure that velocity range is adequate and continuum subtraction ok


In [ ]:
imview(raster   = [{'file':'f'+str(field)+'_13co32.image'}],
        contour = [{'file':'f'+str(field)+'_cont.fits'}])


can't see emission
redo moment0 maps (now excluding noisy first channel!!)
os.system('rm -rf f'+str(field)+'_13co32.mom0*')


In [ ]:
immoments(imagename  = 'f'+str(field)+'_13co32.image',             # CHANGEME (based on above analysis!!)
          outfile    = 'f'+str(field)+'_13co32.mom0',
          moments    = [0],
          includepix = [-10.0,100.0],
          chans      = ('range=[2km/s,6km/s]'))                        # CHANGEME (based on above analysis!!)


export to fits file    


In [ ]:
fbase = 'f'+str(field)+'_13co32.mom0'
os.system('rm -rf '+fbase+'.fits')
exportfits(imagename=fbase,fitsimage=fbase+'.fits')


measure flux


In [ ]:
im_max = imstat(imagename = 'f'+str(field)+'_13co32.mom0')['max'][0]
im_rms = imstat(imagename = 'f'+str(field)+'_13co32.mom0',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
im_flux = imstat(imagename = 'f'+str(field)+'_13co32.mom0',
                 region='circle[['+str(xc)+'pix,'+str(yc)+'pix],'+str(aper)+'arcsec]')['flux'][0]
print 'Flux = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_flux, 1000*im_rms, im_flux/im_rms)


Flux = 91.16 mJy, rms = 33.12 mJy, S/N = 2.8
view continuum and gas


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_13co32.mom0'}],
       contour = [{'file':'f'+str(field)+'_cont.fits'}])


re-center image on source and use measure.py to get COG flux
os.system('rm -rf f'+str(field)+'_13co32.mom0_cropped*')


In [ ]:
ia.fromimage(outfile = 'f'+str(field)+'_13co32.mom0_cropped.image',
             infile  = 'f'+str(field)+'_13co32.mom0.fits',
             region  = box )
ia.close() 
exportfits(imagename = 'f'+str(field)+'_13co32.mom0_cropped.image',
           fitsimage = 'f'+str(field)+'_13co32.mom0_cropped.fits')


In [ ]:
Measuring COG for M/f5_13co32.mom0_cropped.fits
Assuming object center (300.0,300.0)
Background: 0.40 mJy/beam km/s
RMS in annulus 4.0-6.0 arcsec = 32.10 mJy/beam km/s
   i   radius    flux      err       snr
       (asec)    (mJy)     (mJy)
   0     0.10    16.48     8.75      1.9
   1     0.20    33.20    22.55      1.5
   2     0.30    43.70    36.53      1.2 <-- ND
   3     0.40    69.35    41.37      1.7
   4     0.50    91.16    49.68      1.8
   5     0.60    92.98    62.00      1.5
   6     0.70   113.17    61.96      1.8
   7     0.80   143.59    72.87      2.0
   8     0.90   144.24    82.07      1.8
   9     1.00    96.20    90.49      1.1
F = 144.24 mJy
E = 82.07 mJy
S = 1.76
D = 1.80 arcsec


#  C18O line  

don't bother with selfcal as it doesn't help...
os.system('rm -rf f'+str(field)+'_c18o32*')


In [ ]:
clean(vis           = 'f'+str(field)+'.vis.contsub',
      imagename     = 'f'+str(field)+'_c18o32',
      mode          = 'velocity',
      start         = start,
      width         = width,
      nchan         = nchan,
      outframe      = outframe,
      veltype       = veltype,
      restfreq      = '329.33055GHz',
      niter         = 500,
      threshold     = 0,
      interactive   = False,
      mask          = 'f5_cont_mask.crtf',
      imsize        = imsize,
      cell          = cell,
      weighting     ='briggs',
      robust        = robust,
      imagermode    = imagermode)


could not see line clearly, so just cleaned lightly 
within continuum region for all channels at once
export cube to fits file    


In [ ]:
fbase = 'f'+str(field)+'_c18o32'
os.system('rm -rf '+fbase+'.cube.fits')
exportfits(imagename=fbase+'.image',fitsimage=fbase+'.cube.fits')


use viewer to check channel maps and spectrum
make sure that velocity range is adequate and continuum subtraction ok


In [ ]:
imview(raster   = [{'file':'f'+str(field)+'_c18o32.image'}],
        contour = [{'file':'f'+str(field)+'_cont.fits'}])


cannot see line
redo moment0 maps (now excluding noisy first channel!!)
os.system('rm -rf f'+str(field)+'_c18o32.mom0*')


In [ ]:
immoments(imagename  = 'f'+str(field)+'_c18o32.image',            # CHANGEME (based on above analysis!!)
          outfile    = 'f'+str(field)+'_c18o32.mom0',
          moments    = [0],
          includepix = [-10.0,100.0],
          chans      = ('range=[2km/s,6km/s]'))


export to fits file    


In [ ]:
fbase = 'f'+str(field)+'_c18o32.mom0'
os.system('rm -rf '+fbase+'.fits')
exportfits(imagename=fbase,fitsimage=fbase+'.fits')


measure flux


In [ ]:
im_rms = imstat(imagename = 'f'+str(field)+'_c18o32.mom0',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
im_flux = imstat(imagename = 'f'+str(field)+'_c18o32.mom0',
                 region='circle[['+str(xc)+'pix,'+str(yc)+'pix],'+str(aper)+'arcsec]')['flux'][0]
print 'Flux = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_flux, 1000*im_rms, im_flux/im_rms)


Flux = -74.96 mJy, rms = 41.70 mJy, S/N = -1.8
view continuum and gas


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_c18o32.mom0'}],
       contour = [{'file':'f'+str(field)+'_cont.fits'}])


re-center image on source and use measure.py to get COG flux
os.system('rm -rf f'+str(field)+'_c18o32.mom0_cropped*')


In [ ]:
ia.fromimage(outfile = 'f'+str(field)+'_c18o32.mom0_cropped.image',
             infile  = 'f'+str(field)+'_c18o32.mom0.fits',
             region  = box )
ia.close() 
exportfits(imagename = 'f'+str(field)+'_c18o32.mom0_cropped.image',
           fitsimage = 'f'+str(field)+'_c18o32.mom0_cropped.fits')


In [ ]:
Measuring COG for M/f5_c18o32.mom0_cropped.fits
Assuming object center (300.0,300.0)
Background: -0.91 mJy/beam km/s
RMS in annulus 4.0-6.0 arcsec = 41.23 mJy/beam km/s
   i   radius    flux      err       snr
       (asec)    (mJy)     (mJy)
   0     0.10   -15.02    10.54     -1.4
   1     0.20   -48.18    26.01     -1.9
   2     0.30   -80.03    39.82     -2.0 <-- ND
   3     0.40   -80.58    60.88     -1.3
   4     0.50   -74.96    83.07     -0.9
   5     0.60   -79.08    77.65     -1.0
   6     0.70   -84.88   112.52     -0.8
   7     0.80   -85.36   118.79     -0.7
   8     0.90   -97.30   105.26     -0.9
   9     1.00  -104.39   143.24     -0.7
F = -15.02 mJy
E = 10.54 mJy
S = -1.42
D = 0.20 arcsec


#  Image CN  

don't bother with selfcal as it doesn't help...
os.system('rm -rf f'+str(field)+'_cn32_b4sc*')


In [ ]:
clean(vis           = 'f'+str(field)+'.vis.contsub',
      imagename     = 'f'+str(field)+'_cn32_b4sc',
      mode          = 'velocity',
      start         = start,
      width         = width,
      nchan         = nchan,
      outframe      = outframe,
      veltype       = veltype,
      restfreq      = '340.24777GHz',
      niter         = 500,
      threshold     = 0,
      interactive   = False,
      mask          = 'f5_cont_mask.crtf',
      imsize        = imsize,
      cell          = cell,
      weighting     ='briggs',
      robust        = robust,
      imagermode    = imagermode)


could not see line clearly, so just cleaned lightly 
within continuum region for all channels at once
use viewer to check channel maps and spectrum
make sure that velocity range is adequate and continuum subtraction ok


In [ ]:
imview(raster   = [{'file':'f'+str(field)+'_cn32_b4sc.image'}],
        contour = [{'file':'f'+str(field)+'_cont.fits'}])


redo moment0 maps (now excluding noisy first channel!!)
os.system('rm -rf f'+str(field)+'_cn32.mom0*')


In [ ]:
immoments(imagename  = 'f'+str(field)+'_cn32_b4sc.image',            # CHANGEME (based on above analysis!!)
          outfile    = 'f'+str(field)+'_cn32.mom0',
          moments    = [0],
          includepix = [-10.0,100.0],
          chans      = ('range=[1km/s,7km/s]'))


export to fits file    


In [ ]:
fbase = 'f'+str(field)+'_cn32.mom0'
os.system('rm -rf '+fbase+'.fits')
exportfits(imagename=fbase,fitsimage=fbase+'.fits')


measure flux


In [ ]:
im_rms = imstat(imagename = 'f'+str(field)+'_cn32.mom0',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
im_flux = imstat(imagename = 'f'+str(field)+'_cn32.mom0',
                 region='circle[['+str(xc)+'pix,'+str(yc)+'pix],'+str(aper)+'arcsec]')['flux'][0]
print 'Flux = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_flux, 1000*im_rms, im_flux/im_rms)


Flux = 46.73 mJy, rms = 31.35 mJy, S/N = 1.5
view continuum and gas


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_cn32.mom0'}],
       contour = [{'file':'f'+str(field)+'_cont.fits'}])


re-center image on source and use measure.py to get COG flux
os.system('rm -rf f'+str(field)+'_cn32.mom0_cropped*')


In [ ]:
ia.fromimage(outfile = 'f'+str(field)+'_cn32.mom0_cropped.image',
             infile  = 'f'+str(field)+'_cn32.mom0.fits',
             region  = box )
ia.close() 
exportfits(imagename = 'f'+str(field)+'_cn32.mom0_cropped.image',
           fitsimage = 'f'+str(field)+'_cn32.mom0_cropped.fits')


CANNOT SEE EMISSION SO JUST USED 1" APERTURE
      5    0.500     877    48.42   67.352     0.7    78.48   31.629     2.5   0.999
